# Загрузка библиотек для работы

# Загрузка библиотек для работы

In [1]:
# Добавляем библиотеки для работы

In [2]:
import time
import pandas as pd
import numpy as np
import collections
from itertools import chain
import itertools
from scipy.stats import stats
import statistics 
from tapi_yandex_metrika import YandexMetrikaLogsapi
from datetime import date
from datetime import timedelta
import time
import ast 
import re
import os

# Выгрузка данных из Яндекс метрики API

In [3]:
!pip install tapi_yandex_metrika

Defaulting to user installation because normal site-packages is not writeable


In [4]:
!pip install datetime

Defaulting to user installation because normal site-packages is not writeable


In [5]:
ACCESS_TOKEN = "" # вставьте ключ API для своего счетчика ЯМ
COUNTER_ID = "" # укажите номер счетчика Яндекс метрики 

In [6]:
# Get today's date
today = date.today()
# Get 1 days earlier
Yesterday = today - timedelta(days = 1)

client = YandexMetrikaLogsapi(
    access_token= ACCESS_TOKEN,
    default_url_params={'counterId': COUNTER_ID}
)

params = {
"fields":"ym:s:date,ym:s:clientID,ym:s:visitID,ym:s:lastAdvEngine,ym:s:lastReferalSource,ym:s:lastSearchEngineRoot,ym:s:lastSearchEngine,ym:s:lastSocialNetwork,ym:s:lastSocialNetworkProfile,ym:s:lastDirectClickOrder,ym:s:purchaseID,ym:s:purchaseRevenue", 
    "source": "visits",
    "date1": '2025-01-10',
    "date2": '2025-01-15'
}
#year-mounth-day

В LogsAPI нельзя применить фильтрацию перед получением статистики, то есть вам потребуется скачать все сырые данные за указанный период. Далее на своей стороне вы уже выбираете то, что нужно вам.
</br></br>
ym:s:<attribution>TrafficSource - это поле, которое несет в себе источник трафика (Прямой переход, Переход из поисковой системы, Переход из социальной сети и так далее) без детализации. Чтобы получить детализацию, нужно выгрузить еще поля:
</br></br>
<b>Пояснения:</b>
- ym:s:<attribution>AdvEngine - рекламная система, если это был рекламный визит;
- ym:s:<attribution>ReferalSource - сайт, с которого перешли (его адрес), если переход был по ссылкам на сайтах;
- ym:s:<attribution>SearchEngineRoot - какая конкретно была поисковая система, если переход был из поиска;
- ym:s:<attribution>SearchEngine - детализированная информация по поисковой системе, если переход был из поиска;
- ym:s:<attribution>SocialNetwork - соц. сеть, если переход был из социальной сети;
- ym:s:<attribution>SocialNetworkProfile - группа социальной сети, если переход был из социальной сети;
- ym:s:<attribution>DirectClickOrder - кампания Яндекс Директ, если переход был по ней.

In [7]:
# ИСТОЧНИКИ ТРАФИКА НЕ ОЧЕНЬ В ЯМ, ДАЖЕ В СЫРЫХ ДАННЫХ (по описанию выше).
# ЛУЧШЕ МОДЕЛЬ ДЕЛАТЬ ПО ГУГЛ АНАЛИТИКЕ ИЛИ ПО UTM-МЕТКАМ ОРИЕНТИРОВАТЬСЯ.
# Logs api справка: https://yandex.ru/dev/metrika/ru/logs/fields/visits

In [8]:
#Check the possibility of creating a report. Via HTTP GET method.
Check = client.evaluate().get(params=params)
print(Check)
while Check["log_request_evaluation"]["possible"] != True:
 time.sleep(600)
 Check = client.evaluate().get(params=params)

<TapiClient object
{   'log_request_evaluation': {   'expected_size': 5963718,
                                  'log_request_sum_max_size': 10737418240,
                                  'log_request_sum_size': 5255062,
                                  'max_possible_day_quantity': 10797,
                                  'possible': True}}>


In [9]:
# Order a report. Via HTTP POST method.
result = client.create().post(params=params)
request_id = result["log_request"]["request_id"]
print(result)

<TapiClient object
{   'log_request': {   'attribution': 'LASTSIGN',
                       'counter_id': 87875715,
                       'date1': '2025-01-10',
                       'date2': '2025-01-15',
                       'fields': [   'ym:s:date',
                                     'ym:s:clientID',
                                     'ym:s:visitID',
                                     'ym:s:lastAdvEngine',
                                     'ym:s:lastReferalSource',
                                     'ym:s:lastSearchEngineRoot',
                                     'ym:s:lastSearchEngine',
                                     'ym:s:lastSocialNetwork',
                                     'ym:s:lastSocialNetworkProfile',
                                     'ym:s:lastDirectClickOrder',
                                     'ym:s:purchaseID',
                                     'ym:s:purchaseRevenue'],
                       'request_id': 44408016,
                     

In [10]:
info = client.info(requestId=request_id).get()
while info["log_request"]["status"] != "processed":
 time.sleep(10)
 info = client.info(requestId=request_id).get()
 print(info["log_request"]["status"])

created
created
processed


In [11]:
parts = info["log_request"]["parts"]
print(len(parts))

1


In [12]:
for i in range(len(parts)):
    print(i)
    #download from api
    reportTemp = client.download(requestId=request_id, partNumber=0).get()
    #ETL
    df = pd.DataFrame(reportTemp().to_dicts())

0


In [25]:
df

,ym:s:date,ym:s:clientID,ym:s:visitID,ym:s:lastAdvEngine,ym:s:lastReferalSource,ym:s:lastSearchEngineRoot,ym:s:lastSearchEngine,ym:s:lastSocialNetwork,ym:s:lastSocialNetworkProfile,ym:s:lastDirectClickOrder,ym:s:purchaseID,ym:s:purchaseRevenue
0,2025-01-13,1711968971909555406,18296733370605109277,ya_undefined,www.site1.ru,,,,,,[],[]
1,2025-01-13,1731414323281234158,18295331709399335028,ya_undefined,,,,,,,[],[]
2,2025-01-10,1698397648708395672,18224775213957513544,ya_undefined,www.site1.ru,,,,,,[],[]
3,2025-01-14,173641120657364671,18321886933788917833,ya_undefined,,,,,,,[],[]
4,2025-01-14,1721891878245400402,18318870704771301699,ya_undefined,www.site1.ru,,,,,,[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...
64071,2025-01-15,173442009542135044,18348684300527075652,ya_undefined,,,,,,,[],[]
64072,2025-01-14,1711017446799496892,18320901844940357682,ya_undefined,www.site1.ru,,,,,,[],[]
64073,2025-01-10,1734414833390451528,18230597160462188738,ya_undefined,www.site1.ru,,,,,,[],[]
64074,2025-01-15,1716901150747597551,18340271509736259749,ya_undefined,www.site1.ru,,,,,,[],[]


# Предъобработка данных

In [26]:
df.head()

,ym:s:date,ym:s:clientID,ym:s:visitID,ym:s:lastAdvEngine,ym:s:lastReferalSource,ym:s:lastSearchEngineRoot,ym:s:lastSearchEngine,ym:s:lastSocialNetwork,ym:s:lastSocialNetworkProfile,ym:s:lastDirectClickOrder,ym:s:purchaseID,ym:s:purchaseRevenue
0,2025-01-13,1711968971909555406,18296733370605109277,ya_undefined,www.site1.ru,,,,,,[],[]
1,2025-01-13,1731414323281234158,18295331709399335028,ya_undefined,,,,,,,[],[]
2,2025-01-10,1698397648708395672,18224775213957513544,ya_undefined,www.site1.ru,,,,,,[],[]
3,2025-01-14,173641120657364671,18321886933788917833,ya_undefined,,,,,,,[],[]
4,2025-01-14,1721891878245400402,18318870704771301699,ya_undefined,www.site1.ru,,,,,,[],[]


In [27]:
# посчитаем уникальные значения в полях 
unique_values = df['ym:s:lastAdvEngine'].unique()
print(unique_values)

['ya_undefined']


In [28]:
unique_values = df['ym:s:lastReferalSource'].unique()
print(unique_values)
# !

['www.site1.ru' '' 'www.site2.ru' 'yandex.ru' 'ya.ru' 'ntp.msn.com'
 'e.mail.ru' 'www.site3.ru' 'www.site4.ru'
 '6o5pjz4tzmb6t64m_klg_808_5457008bb273c8ffb50ee185278baa69.onlyoffice.disk.yandex.net'
 'google.com' 'statics.teams.cdn.office.net' 'www.site5.ru' 'yandex.by'
 'auth.psbii.lan' 'mail.rostherm.ru' 'touch.kaspersky.com'
 'crm.nawinia.com' 'web.telegram.org' 'portal.tk-kit.net'
 'eks1.bitrix24.ru' 'www.site6.ru']


In [29]:
unique_values = df['ym:s:lastSearchEngineRoot'].unique()
print(unique_values)
# !

['' 'yandex' 'google']


In [30]:
unique_values = df['ym:s:lastSearchEngine'].unique()
print(unique_values)

['' 'yandex_search' 'google_search']


In [31]:
unique_values = df['ym:s:lastSocialNetwork'].unique()
print(unique_values)
# !

['']


In [32]:
unique_values = df['ym:s:lastSocialNetworkProfile'].unique()
print(unique_values)

['']


In [33]:
unique_values = df['ym:s:lastDirectClickOrder'].unique()
print(unique_values)

['']


In [34]:
unique_values = df['ym:s:purchaseID'].unique()
print(unique_values)

['[]' "['15834']" "['15846']" "['15795']" "['15710']" "['15786']"
 "['15726']" "['15720']" "['gaTjFguVXEZyHoP-NTV5azQwpxXGCirp4bAgC8jGL2Q']"
 "['15864']" "['15773']" "['15855']"
 "['pUhkBlgmy1ViNJ9PRILJRXpTgn4_lxW4uWxTJ8LO1K8']" "['15865']" "['15772']"
 "['15709']" "['15743']" "['15767']"
 "['aWId7rtrY9J1qzF9028tfghT4fDRkIvikTn79dRYbx8']" "['15818']" "['15707']"
 "['VU12JBp9ferx-uS0ZGPAI3AhS0NdM9tlArJzHQoNbQc']"
 "['5bJEloxlzSxDtkOoZyBDs1lwbq2LHZH_Sn7dxyPEubc']"
 "['lyyO7qJvD7ECzou2xDUYGpKmVVJCd0DYkDhFYcX8Tj8']" "['15708']"
 "['zkC1VgcbuC5vR6NaSOJLQSaEUkVSvE56wxTtBEAhKgM']" "['15821']" "['15735']"
 "['15742']"
 "['XFEVe5dWrfOR5EFpmbruCO63UXhzMY6BMcv59m3yeEg','DGDsi3N8TBKkCq_IivVl45YpJA9LHeB7-M09n-fQm3M']"
 "['15766']" "['CK0AtXUy8BOVe6GfI_T8hyoAvAXyeb0Lcu80sV-gMxY']" "['15866']"
 "['15837']" "['15807']" "['mQQD6AwbM--DjLLPLoXCqlIulvqSFJRBg_Qfp-VtYa4']"
 "['HD8u9ZStjSZUg8vTDmKRqcyRtzS3n5DHW-NKynoQkHQ']"
 "['1Vgxo6X1aKAol7M9qcfiDTA1s1D5Re1zfDuIIMl-9fI']" "['15854']"
 "['kmIByttgy5HL8dMle

#### Функция для подсчета числовых значений в строке
def count_numeric_ids(purchase_id):
    try:
        # Преобразуем строку в список
        ids = ast.literal_eval(purchase_id)
        # Считаем только числовые значения
        return sum(1 for i in ids if i.isdigit())
    except (ValueError, SyntaxError):
        return 0

#### Применяем функцию к столбцу
df['ym:s:purchaseID'] = df['ym:s:purchaseID'].apply(count_numeric_ids)

In [35]:
# Функция для подсчета всех элементов в списке
def count_values(purchase_id):
    try:
        # Преобразуем строку в список
        ids = ast.literal_eval(purchase_id)
        # Возвращаем количество элементов в списке
        return len(ids)
    except (ValueError, SyntaxError):
        return 0

# Применяем функцию к столбцу (чтобы понять сколько в списке (в строке) было заказов, кол-во)
df['ym:s:purchaseID'] = df['ym:s:purchaseID'].apply(count_values)

In [36]:
df

,ym:s:date,ym:s:clientID,ym:s:visitID,ym:s:lastAdvEngine,ym:s:lastReferalSource,ym:s:lastSearchEngineRoot,ym:s:lastSearchEngine,ym:s:lastSocialNetwork,ym:s:lastSocialNetworkProfile,ym:s:lastDirectClickOrder,ym:s:purchaseID,ym:s:purchaseRevenue
0,2025-01-13,1711968971909555406,18296733370605109277,ya_undefined,www.site1.ru,,,,,,0,[]
1,2025-01-13,1731414323281234158,18295331709399335028,ya_undefined,,,,,,,0,[]
2,2025-01-10,1698397648708395672,18224775213957513544,ya_undefined,www.site1.ru,,,,,,0,[]
3,2025-01-14,173641120657364671,18321886933788917833,ya_undefined,,,,,,,0,[]
4,2025-01-14,1721891878245400402,18318870704771301699,ya_undefined,www.site1.ru,,,,,,0,[]
...,...,...,...,...,...,...,...,...,...,...,...,...
64071,2025-01-15,173442009542135044,18348684300527075652,ya_undefined,,,,,,,0,[]
64072,2025-01-14,1711017446799496892,18320901844940357682,ya_undefined,www.site1.ru,,,,,,0,[]
64073,2025-01-10,1734414833390451528,18230597160462188738,ya_undefined,www.site1.ru,,,,,,0,[]
64074,2025-01-15,1716901150747597551,18340271509736259749,ya_undefined,www.site1.ru,,,,,,0,[]


In [23]:
# какие кол-ва заказов присутствуют во время одного визита:
unique_values = df['ym:s:purchaseID'].unique()
print(unique_values)

[0 1 2 3]


In [37]:
# Рекламы платной нет на проекте, поэтому выбирем только используемые поля:
# Доход в ЭК ЯМ также не передается в данном кейсе. 

In [38]:
df = df[['ym:s:date', 'ym:s:clientID', 'ym:s:visitID', 'ym:s:lastReferalSource', 'ym:s:lastSearchEngineRoot', 'ym:s:lastSocialNetwork', 'ym:s:purchaseID']]

In [39]:
df

,ym:s:date,ym:s:clientID,ym:s:visitID,ym:s:lastReferalSource,ym:s:lastSearchEngineRoot,ym:s:lastSocialNetwork,ym:s:purchaseID
0,2025-01-13,1711968971909555406,18296733370605109277,www.site1.ru,,,0
1,2025-01-13,1731414323281234158,18295331709399335028,,,,0
2,2025-01-10,1698397648708395672,18224775213957513544,www.site1.ru,,,0
3,2025-01-14,173641120657364671,18321886933788917833,,,,0
4,2025-01-14,1721891878245400402,18318870704771301699,www.site1.ru,,,0
...,...,...,...,...,...,...,...
64071,2025-01-15,173442009542135044,18348684300527075652,,,,0
64072,2025-01-14,1711017446799496892,18320901844940357682,www.site1.ru,,,0
64073,2025-01-10,1734414833390451528,18230597160462188738,www.site1.ru,,,0
64074,2025-01-15,1716901150747597551,18340271509736259749,www.site1.ru,,,0


In [41]:
counttt = df['ym:s:purchaseID'].sum()
print(counttt)
# общем кол-во заказов

136


In [42]:
# Заменяем значения, содержащие "yandex" или "google" в поле реферрал, на пустую строку
# убираем все соц сети и поисковые системы из реффералов. Т.к. нам в одной строке нужен только один источник трафика
df = df.copy()
df.loc[df['ym:s:lastReferalSource'].str.contains('yandex|google|ya.ru|android|instagram|inst|facebook|pinterest|bing|vk|mail|yahoo|ok|baidu|ecosia|duckduckgo|odnoklassniki|vkontakte|instagram', case=False, na=False), 'ym:s:lastReferalSource'] = ''


In [43]:
df

,ym:s:date,ym:s:clientID,ym:s:visitID,ym:s:lastReferalSource,ym:s:lastSearchEngineRoot,ym:s:lastSocialNetwork,ym:s:purchaseID
0,2025-01-13,1711968971909555406,18296733370605109277,www.site1.ru,,,0
1,2025-01-13,1731414323281234158,18295331709399335028,,,,0
2,2025-01-10,1698397648708395672,18224775213957513544,www.site1.ru,,,0
3,2025-01-14,173641120657364671,18321886933788917833,,,,0
4,2025-01-14,1721891878245400402,18318870704771301699,www.site1.ru,,,0
...,...,...,...,...,...,...,...
64071,2025-01-15,173442009542135044,18348684300527075652,,,,0
64072,2025-01-14,1711017446799496892,18320901844940357682,www.site1.ru,,,0
64073,2025-01-10,1734414833390451528,18230597160462188738,www.site1.ru,,,0
64074,2025-01-15,1716901150747597551,18340271509736259749,www.site1.ru,,,0


In [44]:
unique_values = df['ym:s:lastReferalSource'].unique()
print(unique_values)

['www.site1.ru' '' 'www.site2.ru' 'ntp.msn.com' 'www.site3.ru'
 'www.site4.ru' 'statics.teams.cdn.office.net' 'www.site5.ru'
 'auth.psbii.lan' 'touch.kaspersky.com' 'crm.nawinia.com'
 'web.telegram.org' 'portal.tk-kit.net' 'eks1.bitrix24.ru' 'www.site6.ru']


In [45]:
unique_values = df['ym:s:lastSearchEngineRoot'].unique()
print(unique_values)

['' 'yandex' 'google']


In [46]:
unique_values = df['ym:s:lastSocialNetwork'].unique()
print(unique_values)

['']


In [47]:
# Добавляем 'search/' в начало каждого значения с использованием loc
df = df.copy()
df.loc[df['ym:s:lastSearchEngineRoot'].str.strip() != '', 'ym:s:lastSearchEngineRoot'] = (
    'search/' + df['ym:s:lastSearchEngineRoot']
)

In [48]:
df

,ym:s:date,ym:s:clientID,ym:s:visitID,ym:s:lastReferalSource,ym:s:lastSearchEngineRoot,ym:s:lastSocialNetwork,ym:s:purchaseID
0,2025-01-13,1711968971909555406,18296733370605109277,www.site1.ru,,,0
1,2025-01-13,1731414323281234158,18295331709399335028,,,,0
2,2025-01-10,1698397648708395672,18224775213957513544,www.site1.ru,,,0
3,2025-01-14,173641120657364671,18321886933788917833,,,,0
4,2025-01-14,1721891878245400402,18318870704771301699,www.site1.ru,,,0
...,...,...,...,...,...,...,...
64071,2025-01-15,173442009542135044,18348684300527075652,,,,0
64072,2025-01-14,1711017446799496892,18320901844940357682,www.site1.ru,,,0
64073,2025-01-10,1734414833390451528,18230597160462188738,www.site1.ru,,,0
64074,2025-01-15,1716901150747597551,18340271509736259749,www.site1.ru,,,0


In [49]:
counttt = df['ym:s:purchaseID'].sum()
print(counttt)
#

136


In [50]:
# Добавляем 'referal/' в начало каждого значения с использованием loc
df = df.copy()
df.loc[df['ym:s:lastReferalSource'].str.strip() != '', 'ym:s:lastReferalSource'] = (
    'referal/' + df['ym:s:lastReferalSource']
)

In [51]:
# Добавляем 'social/' в начало каждого значения с использованием loc
df = df.copy()
df.loc[df['ym:s:lastSocialNetwork'].str.strip() != '', 'ym:s:lastSocialNetwork'] = (
    'social/' + df['ym:s:lastSocialNetwork']
)

In [52]:
df

,ym:s:date,ym:s:clientID,ym:s:visitID,ym:s:lastReferalSource,ym:s:lastSearchEngineRoot,ym:s:lastSocialNetwork,ym:s:purchaseID
0,2025-01-13,1711968971909555406,18296733370605109277,referal/www.site1.ru,,,0
1,2025-01-13,1731414323281234158,18295331709399335028,,,,0
2,2025-01-10,1698397648708395672,18224775213957513544,referal/www.site1.ru,,,0
3,2025-01-14,173641120657364671,18321886933788917833,,,,0
4,2025-01-14,1721891878245400402,18318870704771301699,referal/www.site1.ru,,,0
...,...,...,...,...,...,...,...
64071,2025-01-15,173442009542135044,18348684300527075652,,,,0
64072,2025-01-14,1711017446799496892,18320901844940357682,referal/www.site1.ru,,,0
64073,2025-01-10,1734414833390451528,18230597160462188738,referal/www.site1.ru,,,0
64074,2025-01-15,1716901150747597551,18340271509736259749,referal/www.site1.ru,,,0


In [53]:
unique_values = df['ym:s:lastReferalSource'].unique()
print(unique_values)

['referal/www.site1.ru' '' 'referal/www.site2.ru' 'referal/ntp.msn.com'
 'referal/www.site3.ru' 'referal/www.site4.ru'
 'referal/statics.teams.cdn.office.net' 'referal/www.site5.ru'
 'referal/auth.psbii.lan' 'referal/touch.kaspersky.com'
 'referal/crm.nawinia.com' 'referal/web.telegram.org'
 'referal/portal.tk-kit.net' 'referal/eks1.bitrix24.ru'
 'referal/www.site6.ru']


In [54]:
unique_values = df['ym:s:lastSearchEngineRoot'].unique()
print(unique_values)

['' 'search/yandex' 'search/google']


In [55]:
unique_values = df['ym:s:lastSocialNetwork'].unique()
print(unique_values)

['']


In [56]:
counttt = df['ym:s:purchaseID'].sum()
print(counttt)
#

136


In [57]:
df01=df

In [58]:
df01.head()

,ym:s:date,ym:s:clientID,ym:s:visitID,ym:s:lastReferalSource,ym:s:lastSearchEngineRoot,ym:s:lastSocialNetwork,ym:s:purchaseID
0,2025-01-13,1711968971909555406,18296733370605109277,referal/www.site1.ru,,,0
1,2025-01-13,1731414323281234158,18295331709399335028,,,,0
2,2025-01-10,1698397648708395672,18224775213957513544,referal/www.site1.ru,,,0
3,2025-01-14,173641120657364671,18321886933788917833,,,,0
4,2025-01-14,1721891878245400402,18318870704771301699,referal/www.site1.ru,,,0


In [59]:
# Переименование столбца
df01.rename(columns={'ym:s:clientID': 'user ID'}, inplace=True) # user ID не совсем правильно, надо client ID оставлять название
df01.rename(columns={'ym:s:visitID': 'visitID'}, inplace=True) 
df01.rename(columns={'ym:s:lastReferalSource': 'ReferalSource'}, inplace=True) 
df01.rename(columns={'ym:s:lastSearchEngineRoot': 'SearchEngineRoot'}, inplace=True) 
df01.rename(columns={'ym:s:lastSocialNetwork': 'SocialNetwork'}, inplace=True) 
df01.rename(columns={'ym:s:purchaseID': 'conversions'}, inplace=True) 


In [60]:
df01.head()

,ym:s:date,user ID,visitID,ReferalSource,SearchEngineRoot,SocialNetwork,conversions
0,2025-01-13,1711968971909555406,18296733370605109277,referal/www.site1.ru,,,0
1,2025-01-13,1731414323281234158,18295331709399335028,,,,0
2,2025-01-10,1698397648708395672,18224775213957513544,referal/www.site1.ru,,,0
3,2025-01-14,173641120657364671,18321886933788917833,,,,0
4,2025-01-14,1721891878245400402,18318870704771301699,referal/www.site1.ru,,,0


In [61]:
# Удаляем столбец ym:s:date
df01.drop(columns=['ym:s:date'], inplace=True)

In [62]:
df01.head()

,user ID,visitID,ReferalSource,SearchEngineRoot,SocialNetwork,conversions
0,1711968971909555406,18296733370605109277,referal/www.site1.ru,,,0
1,1731414323281234158,18295331709399335028,,,,0
2,1698397648708395672,18224775213957513544,referal/www.site1.ru,,,0
3,173641120657364671,18321886933788917833,,,,0
4,1721891878245400402,18318870704771301699,referal/www.site1.ru,,,0


In [63]:
counttt = df01['conversions'].sum()
print(counttt)
#

136


# Модель цепи Маркова

In [50]:
#Значение доходности не было в Электронной коммерции Яндекс метрики, поэтому делаем только по purchase. 

In [64]:
# Цепи Маркова могут сформировать только гипотезы, которые нужно проверять. Т.к. множество факторов, оказывающих влияние на результат
# Если vk РК была с акцией, а остальные нет. И vk по результатом модели имеет больший revenue, чем без
# ,то мы не можем присвоить этому источнику больше конверсии или дохода. Т.к. у источника vk была не честная конкуренция. 

In [65]:
df01=df

In [66]:
df01

,user ID,visitID,ReferalSource,SearchEngineRoot,SocialNetwork,conversions
0,1711968971909555406,18296733370605109277,referal/www.site1.ru,,,0
1,1731414323281234158,18295331709399335028,,,,0
2,1698397648708395672,18224775213957513544,referal/www.site1.ru,,,0
3,173641120657364671,18321886933788917833,,,,0
4,1721891878245400402,18318870704771301699,referal/www.site1.ru,,,0
...,...,...,...,...,...,...
64071,173442009542135044,18348684300527075652,,,,0
64072,1711017446799496892,18320901844940357682,referal/www.site1.ru,,,0
64073,1734414833390451528,18230597160462188738,referal/www.site1.ru,,,0
64074,1716901150747597551,18340271509736259749,referal/www.site1.ru,,,0


In [67]:
# Создание нового поля с учетом условия
df01['medium/source'] = df01[['ReferalSource', 'SearchEngineRoot', 'SocialNetwork']] \
    .apply(lambda row: ' / '.join(row.dropna()) if row.dropna().any() else 'DirectOrIndefinite', axis=1)


In [68]:
df01

,user ID,visitID,ReferalSource,SearchEngineRoot,SocialNetwork,conversions,medium/source
0,1711968971909555406,18296733370605109277,referal/www.site1.ru,,,0,referal/www.site1.ru / /
1,1731414323281234158,18295331709399335028,,,,0,DirectOrIndefinite
2,1698397648708395672,18224775213957513544,referal/www.site1.ru,,,0,referal/www.site1.ru / /
3,173641120657364671,18321886933788917833,,,,0,DirectOrIndefinite
4,1721891878245400402,18318870704771301699,referal/www.site1.ru,,,0,referal/www.site1.ru / /
...,...,...,...,...,...,...,...
64071,173442009542135044,18348684300527075652,,,,0,DirectOrIndefinite
64072,1711017446799496892,18320901844940357682,referal/www.site1.ru,,,0,referal/www.site1.ru / /
64073,1734414833390451528,18230597160462188738,referal/www.site1.ru,,,0,referal/www.site1.ru / /
64074,1716901150747597551,18340271509736259749,referal/www.site1.ru,,,0,referal/www.site1.ru / /


In [69]:
# Если в конце строки есть " /", удаляем только его
df01['medium/source'] = df01['medium/source'].str.rstrip(' /')  # Убираем "//" в конце

In [70]:
df01

,user ID,visitID,ReferalSource,SearchEngineRoot,SocialNetwork,conversions,medium/source
0,1711968971909555406,18296733370605109277,referal/www.site1.ru,,,0,referal/www.site1.ru
1,1731414323281234158,18295331709399335028,,,,0,DirectOrIndefinite
2,1698397648708395672,18224775213957513544,referal/www.site1.ru,,,0,referal/www.site1.ru
3,173641120657364671,18321886933788917833,,,,0,DirectOrIndefinite
4,1721891878245400402,18318870704771301699,referal/www.site1.ru,,,0,referal/www.site1.ru
...,...,...,...,...,...,...,...
64071,173442009542135044,18348684300527075652,,,,0,DirectOrIndefinite
64072,1711017446799496892,18320901844940357682,referal/www.site1.ru,,,0,referal/www.site1.ru
64073,1734414833390451528,18230597160462188738,referal/www.site1.ru,,,0,referal/www.site1.ru
64074,1716901150747597551,18340271509736259749,referal/www.site1.ru,,,0,referal/www.site1.ru


In [71]:
counttt = df01['conversions'].sum()
print(counttt)
#

136


In [72]:
# Оставляем только нужные столбцы
df02 = df01[['visitID', 'user ID', 'medium/source', 'conversions']]

In [73]:
df02 = df02.copy()
df02.loc[:, 'conversions'] = df02['conversions'].fillna(0)
# df02.loc[:, 'revenue'] = df02['revenue'].fillna(0)

In [74]:
df02

,visitID,user ID,medium/source,conversions
0,18296733370605109277,1711968971909555406,referal/www.site1.ru,0
1,18295331709399335028,1731414323281234158,DirectOrIndefinite,0
2,18224775213957513544,1698397648708395672,referal/www.site1.ru,0
3,18321886933788917833,173641120657364671,DirectOrIndefinite,0
4,18318870704771301699,1721891878245400402,referal/www.site1.ru,0
...,...,...,...,...
64071,18348684300527075652,173442009542135044,DirectOrIndefinite,0
64072,18320901844940357682,1711017446799496892,referal/www.site1.ru,0
64073,18230597160462188738,1734414833390451528,referal/www.site1.ru,0
64074,18340271509736259749,1716901150747597551,referal/www.site1.ru,0


In [75]:
# df02['conversions'] = df02['conversions'].astype(int)   # df02['conversions'] и так тип значения int
# df02['revenue'] = df02['revenue'].astype(int)

In [76]:
counttt = df02['conversions'].sum()
print(counttt)
#

136


In [77]:
#агрегируем конверсии или доход (revenue) и кол-во пользователей по пути каналу/источнику трафика (с шагами)
#Каждый шаг по визиту пользователя разделяем знаком: >
df03 = df02.groupby('user ID').agg({
    'medium/source': lambda x: ' > '.join(
        key for i, key in enumerate(list(x)) if i == 0 or key != list(x)[i - 1]
    ),
    'conversions': 'sum',
    #'revenue': 'sum',
    'visitID': 'count'
}).reset_index()

In [78]:
df03

,user ID,medium/source,conversions,visitID
0,1644890525882035644,referal/www.site1.ru,0,1
1,1647548307607766102,DirectOrIndefinite,0,1
2,1647558797511737211,referal/www.site1.ru,0,4
3,1647560220541847207,referal/www.site1.ru,0,1
4,16475692801023569692,referal/www.site1.ru,0,11
...,...,...,...,...
17124,173696461694087817,referal/www.site1.ru,0,1
17125,1736967318616024606,DirectOrIndefinite,0,1
17126,1736970562544444024,/ search/yandex,0,1
17127,1736973048772231573,DirectOrIndefinite,0,1


In [79]:
# Группировка по полю 'medium/source' и подсчет суммы по 'conversions'
df03_grouped = df03.groupby('medium/source')['conversions'].sum().reset_index()

# Фильтрация строк, где 'conversions' не равен 0, и сортировка по убыванию
df03_grouped = df03_grouped[df03_grouped['conversions'] > 0].sort_values(by='conversions', ascending=False)

In [80]:
df03_grouped

,medium/source,conversions
22,DirectOrIndefinite,51
74,referal/www.site1.ru,35
108,referal/www.site2.ru,9
35,DirectOrIndefinite > referal/www.site1.ru > Di...,6
82,referal/www.site1.ru > DirectOrIndefinite,6
4,/ search/yandex,5
119,referal/www.site2.ru > referal/www.site1.ru,2
133,referal/www.site4.ru > referal/www.site1.ru,2
32,DirectOrIndefinite > referal/www.site1.ru,2
33,DirectOrIndefinite > referal/www.site1.ru > /...,1


In [81]:
counttt = df03['conversions'].sum()
print(counttt)
#

136


In [82]:
# Группировка по "medium/source" и агрегация суммой
#df04 = df03.groupby("medium/source")[["conversions", "revenue"]].sum().reset_index()
df04 = df03.groupby("medium/source").agg(
    conversions=("conversions", "sum"),
    #revenue=("revenue", "sum"),
    sessions=("visitID", "sum"),
    users=("user ID", "count")
).reset_index()

In [83]:
df04

,medium/source,conversions,sessions,users
0,/ search/google,0,29,11
1,/ search/google > DirectOrIndefinite > / sea...,1,10,1
2,/ search/google > referal/www.site1.ru,0,2,1
3,/ search/google > referal/www.site1.ru > / s...,0,8,1
4,/ search/yandex,5,1166,417
...,...,...,...,...
133,referal/www.site4.ru > referal/www.site1.ru,2,19,4
134,referal/www.site4.ru > referal/www.site2.ru > ...,0,7,1
135,referal/www.site4.ru > referal/www.site5.ru > ...,0,10,1
136,referal/www.site5.ru,0,1,1


In [84]:
counttt = df04['conversions'].sum()
print(counttt)
#

136


In [85]:
# Сортировка для удобства (по conversions). Можно делать и по revenue
df05 = df04.sort_values(by="conversions", ascending=False)
df05 = df05.rename(columns={"medium/source": "path"})

In [86]:
df05

,path,conversions,sessions,users
22,DirectOrIndefinite,51,32918,8802
74,referal/www.site1.ru,35,19118,5485
108,referal/www.site2.ru,9,3457,1171
35,DirectOrIndefinite > referal/www.site1.ru > Di...,6,1529,203
82,referal/www.site1.ru > DirectOrIndefinite,6,662,158
...,...,...,...,...
47,DirectOrIndefinite > referal/www.site2.ru,0,59,20
46,DirectOrIndefinite > referal/www.site1.ru > re...,0,4,1
45,DirectOrIndefinite > referal/www.site1.ru > re...,0,12,2
44,DirectOrIndefinite > referal/www.site1.ru > Di...,0,10,1


In [87]:
counttt = df05['conversions'].sum()
print(counttt)
#

136


In [88]:
import_dataset=df05

In [89]:
import_dataset.head()

,path,conversions,sessions,users
22,DirectOrIndefinite,51,32918,8802
74,referal/www.site1.ru,35,19118,5485
108,referal/www.site2.ru,9,3457,1171
35,DirectOrIndefinite > referal/www.site1.ru > Di...,6,1529,203
82,referal/www.site1.ru > DirectOrIndefinite,6,662,158


In [90]:
# считаем ass_conversion (оценка вклада каналов в конверсию на основе цепей Маркова)

In [91]:
def unique(list1):  
    unique_list = []   
    for x in list1: 
        if x not in unique_list: 
            unique_list.append(x) 

    return(unique_list)

def split_fun(path):
    return path.split('>')

def calculate_rank(vector):
  a={}
  rank=0
  for num in sorted(vector):
    if num not in a:
      a[num]=rank
      rank=rank+1
  return[a[i] for i in vector]

def transition_matrix_func(import_data):

    z_import_data=import_data.copy()

    z_import_data['path1']='start>'+z_import_data['path']
    z_import_data['path2']=z_import_data['path1']+'>convert'


    z_import_data['pair']=z_import_data['path2'].apply(split_fun)

    zlist=z_import_data['pair'].tolist()
    zlist=list(chain.from_iterable(zlist))
    zlist=list(map(str.strip, zlist))
    T=calculate_rank(zlist)

    M = [[0]*len(unique(zlist)) for _ in range(len(unique(zlist)))]

    for (i,j) in zip(T,T[1:]):
        M[i][j] += 1

    x_df=pd.DataFrame(M)

    np.fill_diagonal(x_df.values,0)

    x_df=pd.DataFrame(x_df.values/x_df.values.sum(axis=1)[:,None])
    x_df.columns=sorted(unique(zlist))
    x_df['index']=sorted(unique(zlist))
    x_df.set_index("index", inplace = True) 
    x_df.loc['convert',:]=0
    return(x_df)

def simulation(trans,n):

    sim=['']*n
    sim[0]= 'start'
    i=1
    while i<n:
        sim[i] = np.random.choice(trans.columns, 1, p=trans.loc[sim[i-1],:])[0]
        if sim[i]=='convert':
            break
        i=i+1

    return sim[0:i+1]


def markov_chain(data_set,no_iteration=10,no_of_simulation=10000,alpha=5):


    import_dataset_v1=data_set.copy()
    import_dataset_v1=(import_dataset_v1.reindex(import_dataset_v1.index.repeat(import_dataset_v1.conversions))).reset_index()
    import_dataset_v1['conversions']=1

    import_dataset_v1=import_dataset_v1[['path','conversions']]

    import_dataset=(import_dataset_v1.groupby(['path']).sum()).reset_index()
    import_dataset['probability']=import_dataset['conversions']/import_dataset['conversions'].sum()

    final=pd.DataFrame()


    for k in range(0,no_iteration):
        start = time.time()
        import_data=pd.DataFrame({'path':np.random.choice(import_dataset['path'],size=import_dataset['conversions'].sum(),p=import_dataset['probability'],replace=True)})
        import_data['conversions']=1                           

        tr_matrix=transition_matrix_func(import_data)
        channel_only = list(filter(lambda k0: k0 not in ['start','convert'], tr_matrix.columns)) 

        ga_ex=pd.DataFrame()
        tr_mat=tr_matrix.copy()
        p=[]

        i=0
        while i<no_of_simulation:
            p.append(unique(simulation(tr_mat,1000)))
            i=i+1


        path=list(itertools.chain.from_iterable(p))
        counter=collections.Counter(path)

        df=pd.DataFrame({'path':list(counter.keys()),'count':list(counter.values())})
        df=df[['path','count']]
        ###ga_ex=ga_ex.append(df,ignore_index=True) 
        ga_ex = pd.concat([ga_ex, df], ignore_index=True)

        df1=(pd.DataFrame(ga_ex.groupby(['path'])[['count']].sum())).reset_index()

        df1['removal_effects']=df1['count']/len(path)
        #df1['removal_effects']=df1['count']/sum(df1['count'][df1['path']=='convert'])
        df1=df1[df1['path'].isin(channel_only)]
        df1['ass_conversion']=df1['removal_effects']/sum(df1['removal_effects'])

        df1['ass_conversion']=df1['ass_conversion']*sum(import_dataset['conversions']) 

        ###final=final.append(df1,ignore_index=True)
        final = pd.concat([final, df1], ignore_index=True)
        end = time.time()
        t1=(end - start)
        print(t1)   

    '''
    H0: u=0
    H1: u>0
    '''


    unique_channel=unique(final['path'])
    #final=(pd.DataFrame(final.groupby(['path'])[['ass_conversion']].mean())).reset_index()
    final_df=pd.DataFrame()

    for i in range(0,len(unique_channel)):

        x=(final['ass_conversion'][final['path']==unique_channel[i]]).values
        final_df.loc[i,0]=unique_channel[i]
        final_df.loc[i,1]=x.mean()

        v=stats.ttest_1samp(x,0)
        final_df.loc[i,2]=v[1]/2

        if v[1]/2<=alpha/100:
            final_df.loc[i,3]=str(100-alpha)+'% statistically confidence'
        else:
            final_df.loc[i,3]=str(100-alpha)+'% statistically not confidence'

        final_df.loc[i,4]=len(x)
        final_df.loc[i,5]=statistics.stdev(x)
        final_df.loc[i,6]=v[0]

    final_df.columns=['channel','ass_conversion','p_value','confidence_status','frequency','standard_deviation','t_statistics']       
    final_df['ass_conversion']=sum(import_dataset['conversions']) *final_df['ass_conversion'] /sum(final_df['ass_conversion'])

    return final_df,final

import_dataset=df05

data_conversion,dataset=markov_chain(import_dataset,no_iteration=10,no_of_simulation=10000,alpha=5)

2.4659171104431152
2.4124276638031006
2.4505112171173096
2.3878073692321777
2.538325309753418
2.537050724029541
2.3866801261901855
2.5184714794158936
2.494791269302368
2.7511894702911377


In [92]:
data_conversion, dataset = markov_chain(import_dataset, no_iteration=10, no_of_simulation=10000, alpha=5)

# Посмотрим итоговую таблицу
print(data_conversion.head())

2.586439371109009
2.5745396614074707
2.6330196857452393
2.517282247543335
2.5728485584259033
2.4964444637298584
2.5171966552734375
2.348747491836548
2.648725986480713
2.585252523422241
                channel  ass_conversion       p_value  \
0       / search/google        3.015256  1.345423e-03   
1       / search/yandex        7.485237  9.902638e-08   
2    DirectOrIndefinite       59.224519  1.666295e-13   
3  referal/www.site1.ru       50.973597  5.048747e-12   
4  referal/www.site2.ru       12.973493  6.134015e-08   

              confidence_status  frequency  standard_deviation  t_statistics  
0  95% statistically confidence        6.0            1.352131      5.511246  
1  95% statistically confidence       10.0            1.699000     14.056622  
2  95% statistically confidence       10.0            3.010152     62.774282  
3  95% statistically confidence       10.0            3.789200     42.920668  
4  95% statistically confidence       10.0            2.786313     14.855789 

In [93]:
data_conversion

,channel,ass_conversion,p_value,confidence_status,frequency,standard_deviation,t_statistics
0,/ search/google,3.015256,1.345423e-03,95% statistically confidence,6.0,1.352131,5.511246
1,/ search/yandex,7.485237,9.902638e-08,95% statistically confidence,10.0,1.699000,14.056622
2,DirectOrIndefinite,59.224519,1.666295e-13,95% statistically confidence,10.0,3.010152,62.774282
3,referal/www.site1.ru,50.973597,5.048747e-12,95% statistically confidence,10.0,3.789200,42.920668
4,referal/www.site2.ru,12.973493,6.134015e-08,95% statistically confidence,10.0,2.786313,14.855789
5,referal/www.site4.ru,2.327897,4.610840e-06,95% statistically confidence,10.0,0.833072,8.915586


In [94]:
# Функция для очистки значений
def clean_medium_source(value):
    value = re.sub(r'^[^a-zA-Z]+', '', value)
    return value

# Применяем очистку к столбцу medium/source
data_conversion['channel'] = data_conversion['channel'].apply(clean_medium_source)

In [95]:
data_conversion

,channel,ass_conversion,p_value,confidence_status,frequency,standard_deviation,t_statistics
0,search/google,3.015256,1.345423e-03,95% statistically confidence,6.0,1.352131,5.511246
1,search/yandex,7.485237,9.902638e-08,95% statistically confidence,10.0,1.699000,14.056622
2,DirectOrIndefinite,59.224519,1.666295e-13,95% statistically confidence,10.0,3.010152,62.774282
3,referal/www.site1.ru,50.973597,5.048747e-12,95% statistically confidence,10.0,3.789200,42.920668
4,referal/www.site2.ru,12.973493,6.134015e-08,95% statistically confidence,10.0,2.786313,14.855789
5,referal/www.site4.ru,2.327897,4.610840e-06,95% statistically confidence,10.0,0.833072,8.915586


# Промежуточная итоговая таблица (user, sessions, convertions by path)


In [96]:
# группируем по medium/source итоговые значения (чтобы оценить вклад не пути, а конкретного канала/источника)
result1 = (
    df01.groupby('medium/source')
    .agg(
        visitID_count=('visitID', 'count'),
        unique_user_count=('user ID', 'nunique')
    )
    .reset_index()
)

In [97]:
result1

,medium/source,visitID_count,unique_user_count
0,/ search/google,48,16
1,/ search/yandex,1408,542
2,DirectOrIndefinite,36511,9783
3,referal/auth.psbii.lan,5,1
4,referal/crm.nawinia.com,1,1
5,referal/eks1.bitrix24.ru,1,1
6,referal/ntp.msn.com,162,40
7,referal/portal.tk-kit.net,1,1
8,referal/statics.teams.cdn.office.net,6,3
9,referal/touch.kaspersky.com,3,1


In [98]:
# Функция для очистки значений
def clean_medium_source(value):
    value = re.sub(r'^[^a-zA-Z]+', '', value)
    return value

# Применяем очистку к столбцу medium/source
result1['medium/source'] = result1['medium/source'].apply(clean_medium_source)

In [99]:
result1

,medium/source,visitID_count,unique_user_count
0,search/google,48,16
1,search/yandex,1408,542
2,DirectOrIndefinite,36511,9783
3,referal/auth.psbii.lan,5,1
4,referal/crm.nawinia.com,1,1
5,referal/eks1.bitrix24.ru,1,1
6,referal/ntp.msn.com,162,40
7,referal/portal.tk-kit.net,1,1
8,referal/statics.teams.cdn.office.net,6,3
9,referal/touch.kaspersky.com,3,1


In [100]:
# Оставляем только нужные столбцы из data_conversion
data_conversion_subset = data_conversion[['channel', 'ass_conversion']]

# Выполняем объединение (merge)
result02 = result1.merge(
    data_conversion_subset,
    left_on='medium/source',  # Поле из первой таблицы
    right_on='channel',       # Поле из второй таблицы
    how='left'                # Тип объединения: left join
)


In [101]:
result02

,medium/source,visitID_count,unique_user_count,channel,ass_conversion
0,search/google,48,16,search/google,3.015256
1,search/yandex,1408,542,search/yandex,7.485237
2,DirectOrIndefinite,36511,9783,DirectOrIndefinite,59.224519
3,referal/auth.psbii.lan,5,1,NaN,NaN
4,referal/crm.nawinia.com,1,1,NaN,NaN
5,referal/eks1.bitrix24.ru,1,1,NaN,NaN
6,referal/ntp.msn.com,162,40,NaN,NaN
7,referal/portal.tk-kit.net,1,1,NaN,NaN
8,referal/statics.teams.cdn.office.net,6,3,NaN,NaN
9,referal/touch.kaspersky.com,3,1,NaN,NaN


In [102]:
# Удаляем столбец channel, если он больше не нужен
result02 = result02.drop(columns=['channel'])

# Сортировка по полю ass_conversion от большего к меньшему
result02 = result02.sort_values(by='ass_conversion', ascending=False)

In [103]:
# Замена NaN на 0 в поле ass_conversion
result02['ass_conversion'] = result02['ass_conversion'].fillna(0)


In [104]:
result02

,medium/source,visitID_count,unique_user_count,ass_conversion
2,DirectOrIndefinite,36511,9783,59.224519
11,referal/www.site1.ru,21673,6460,50.973597
12,referal/www.site2.ru,4043,1402,12.973493
1,search/yandex,1408,542,7.485237
0,search/google,48,16,3.015256
14,referal/www.site4.ru,185,74,2.327897
3,referal/auth.psbii.lan,5,1,0.000000
4,referal/crm.nawinia.com,1,1,0.000000
5,referal/eks1.bitrix24.ru,1,1,0.000000
6,referal/ntp.msn.com,162,40,0.000000


<b>Доп Revenue</b>

In [81]:
# цепь Маркова отдельно нужно построить и для revenue. И подтянуть значение ass_revenue

In [82]:
#ниже код в Markdown

In [105]:
# считаем ass_revenue (для других Я.метрик, где есть Электронная коммерция с доходом). 
# Затем значения можно присоединить к основной итоговой таблице
# в нашем кейсе в ЭК ЯМ нет доходности (руб)

def unique(list1):  
    unique_list = []   
    for x in list1: 
        if x not in unique_list: 
            unique_list.append(x) 

    return(unique_list)

def split_fun(path):
    return path.split('>')

def calculate_rank(vector):
  a={}
  rank=0
  for num in sorted(vector):
    if num not in a:
      a[num]=rank
      rank=rank+1
  return[a[i] for i in vector]

def transition_matrix_func(import_data):

    z_import_data=import_data.copy()

    z_import_data['path1']='start>'+z_import_data['path']
    z_import_data['path2']=z_import_data['path1']+'>convert'


    z_import_data['pair']=z_import_data['path2'].apply(split_fun)

    zlist=z_import_data['pair'].tolist()
    zlist=list(chain.from_iterable(zlist))
    zlist=list(map(str.strip, zlist))
    T=calculate_rank(zlist)

    M = [[0]*len(unique(zlist)) for _ in range(len(unique(zlist)))]

    for (i,j) in zip(T,T[1:]):
        M[i][j] += 1

    x_df=pd.DataFrame(M)

    np.fill_diagonal(x_df.values,0)

    x_df=pd.DataFrame(x_df.values/x_df.values.sum(axis=1)[:,None])
    x_df.columns=sorted(unique(zlist))
    x_df['index']=sorted(unique(zlist))
    x_df.set_index("index", inplace = True) 
    x_df.loc['convert',:]=0
    return(x_df)

def simulation(trans,n):

    sim=['']*n
    sim[0]= 'start'
    i=1
    while i<n:
        sim[i] = np.random.choice(trans.columns, 1, p=trans.loc[sim[i-1],:])[0]
        if sim[i]=='convert':
            break
        i=i+1

    return sim[0:i+1]


def markov_chain(data_set,no_iteration=10,no_of_simulation=10000,alpha=5):


    import_dataset_v1=data_set.copy()
    import_dataset_v1=(import_dataset_v1.reindex(import_dataset_v1.index.repeat(import_dataset_v1.revenue))).reset_index()
    import_dataset_v1['revenue']=1

    import_dataset_v1=import_dataset_v1[['path','revenue']]

    import_dataset=(import_dataset_v1.groupby(['path']).sum()).reset_index()
    import_dataset['probability']=import_dataset['revenue']/import_dataset['revenue'].sum()

    final=pd.DataFrame()


    for k in range(0,no_iteration):
        start = time.time()
        import_data=pd.DataFrame({'path':np.random.choice(import_dataset['path'],size=import_dataset['revenue'].sum(),p=import_dataset['probability'],replace=True)})
        import_data['revenue']=1                           

        tr_matrix=transition_matrix_func(import_data)
        channel_only = list(filter(lambda k0: k0 not in ['start','convert'], tr_matrix.columns)) 

        ga_ex=pd.DataFrame()
        tr_mat=tr_matrix.copy()
        p=[]

        i=0
        while i<no_of_simulation:
            p.append(unique(simulation(tr_mat,1000)))
            i=i+1


        path=list(itertools.chain.from_iterable(p))
        counter=collections.Counter(path)

        df=pd.DataFrame({'path':list(counter.keys()),'count':list(counter.values())})
        df=df[['path','count']]
        ###ga_ex=ga_ex.append(df,ignore_index=True) 
        ga_ex = pd.concat([ga_ex, df], ignore_index=True)

        df1=(pd.DataFrame(ga_ex.groupby(['path'])[['count']].sum())).reset_index()

        df1['removal_effects']=df1['count']/len(path)
        #df1['removal_effects']=df1['count']/sum(df1['count'][df1['path']=='convert'])
        df1=df1[df1['path'].isin(channel_only)]
        df1['ass_revenue']=df1['removal_effects']/sum(df1['removal_effects'])

        df1['ass_revenue']=df1['ass_revenue']*sum(import_dataset['revenue']) 

        ###final=final.append(df1,ignore_index=True)
        final = pd.concat([final, df1], ignore_index=True)
        end = time.time()
        t1=(end - start)
        print(t1)   

    '''
    H0: u=0
    H1: u>0
    '''


    unique_channel=unique(final['path'])
    #final=(pd.DataFrame(final.groupby(['path'])[['ass_conversion']].mean())).reset_index()
    final_df=pd.DataFrame()

    for i in range(0,len(unique_channel)):

        x=(final['ass_revenue'][final['path']==unique_channel[i]]).values
        final_df.loc[i,0]=unique_channel[i]
        final_df.loc[i,1]=x.mean()

        v=stats.ttest_1samp(x,0)
        final_df.loc[i,2]=v[1]/2

        if v[1]/2<=alpha/100:
            final_df.loc[i,3]=str(100-alpha)+'% statistically confidence'
        else:
            final_df.loc[i,3]=str(100-alpha)+'% statistically not confidence'

        final_df.loc[i,4]=len(x)
        final_df.loc[i,5]=statistics.stdev(x)
        final_df.loc[i,6]=v[0]

    final_df.columns=['channel','ass_revenue','p_value','confidence_status','frequency','standard_deviation','t_statistics']       
    final_df['ass_revenue']=sum(import_dataset['revenue']) *final_df['ass_revenue'] /sum(final_df['ass_revenue'])

    return final_df,final

import_dataset=df05

data_revenue,dataset=markov_chain(import_dataset,no_iteration=10,no_of_simulation=10000,alpha=5)

data_revenue, dataset = markov_chain(import_dataset, no_iteration=10, no_of_simulation=10000, alpha=5)

#### Посмотрим итоговую таблицу
print(data_revenue.head())

data_revenue

# Итоговая таблица (с расходами)

In [107]:
#подгружаем Эксель таблицу с расходами за тот же периож (таблица для примера, с случайными значениями)
expenses=pd.read_csv('C:/Users/Павел/Desktop/ЯндексПрактикум/expenses.csv', sep=';')

In [108]:
expenses

,path,"expenses, rub"
0,search/yandex,1000000
1,search/google,500000


In [109]:
# Выполняем объединение (merge)
result03 = result02.merge(
    expenses,
    left_on='medium/source',  # Поле из первой таблицы
    right_on='path',       # Поле из второй таблицы
    how='left'                # Тип объединения: left join
)


In [111]:
result03.drop('path', axis=1, inplace=True)
# удаляем столбец path 

In [112]:
result03

,medium/source,visitID_count,unique_user_count,ass_conversion,"expenses, rub"
0,DirectOrIndefinite,36511,9783,59.224519,NaN
1,referal/www.site1.ru,21673,6460,50.973597,NaN
2,referal/www.site2.ru,4043,1402,12.973493,NaN
3,search/yandex,1408,542,7.485237,1000000.0
4,search/google,48,16,3.015256,500000.0
5,referal/www.site4.ru,185,74,2.327897,NaN
6,referal/auth.psbii.lan,5,1,0.000000,NaN
7,referal/crm.nawinia.com,1,1,0.000000,NaN
8,referal/eks1.bitrix24.ru,1,1,0.000000,NaN
9,referal/ntp.msn.com,162,40,0.000000,NaN


In [113]:
# ЕСЛИ ИЗ ЯМ ПОДТЯНУТЬ ДОХОДНОСТЬ, БУДЕТ НАГЛЯДНО ПОНЯТНО, ОКУПИЛСЯ ЛИ РЕКЛАМНЫЙ КАНАЛ (рк) ЗА ИССЛЕДУЕМЫЙ ПЕРИОД (БЕЗ УЧЕТА ltv)

In [114]:
# Замена NaN на 0 
result03['expenses, rub'] = result03['expenses, rub'].fillna(0)

In [115]:
result03

,medium/source,visitID_count,unique_user_count,ass_conversion,"expenses, rub"
0,DirectOrIndefinite,36511,9783,59.224519,0.0
1,referal/www.site1.ru,21673,6460,50.973597,0.0
2,referal/www.site2.ru,4043,1402,12.973493,0.0
3,search/yandex,1408,542,7.485237,1000000.0
4,search/google,48,16,3.015256,500000.0
5,referal/www.site4.ru,185,74,2.327897,0.0
6,referal/auth.psbii.lan,5,1,0.000000,0.0
7,referal/crm.nawinia.com,1,1,0.000000,0.0
8,referal/eks1.bitrix24.ru,1,1,0.000000,0.0
9,referal/ntp.msn.com,162,40,0.000000,0.0


# Выгружаем таблицу в Excel

In [116]:
# Создать папку Markov на диске C, если она не существует
path = 'C:/Markov'
os.makedirs(path, exist_ok=True)

# Сохранить данные таблицы result03 в CSV файл
result03.to_excel('C:/Markov/Markov1.xlsx', index=False)


# Чистка api запросов (обязательно в конце)

Чистим api запрос

In [117]:
!pip install tapi_yandex_metrika

Defaulting to user installation because normal site-packages is not writeable


In [118]:
!pip install datetime

Defaulting to user installation because normal site-packages is not writeable


In [119]:
ACCESS_TOKEN = "" # вставьте ключ API для своего счетчика ЯМ
COUNTER_ID = "" # укажите номер счетчика Яндекс метрики 

In [120]:
client = YandexMetrikaLogsapi(
    access_token= ACCESS_TOKEN,
    default_url_params={'counterId': COUNTER_ID}
)

In [121]:
result = client.allinfo().get()
print(result)

<TapiClient object
{   'requests': [   {   'attribution': 'LASTSIGN',
                        'counter_id': 87875715,
                        'date1': '2025-01-10',
                        'date2': '2025-01-15',
                        'fields': [   'ym:s:date',
                                      'ym:s:clientID',
                                      'ym:s:visitID',
                                      'ym:s:lastAdvEngine',
                                      'ym:s:lastReferalSource',
                                      'ym:s:lastSearchEngineRoot',
                                      'ym:s:lastSearchEngine',
                                      'ym:s:lastSocialNetwork',
                                      'ym:s:lastSocialNetworkProfile',
                                      'ym:s:lastDirectClickOrder',
                                      'ym:s:purchaseID',
                                      'ym:s:purchaseRevenue'],
                        'parts': [{'part_number': 0,

In [ ]:
result = client.clean(requestId=40374631).post()
print(result)

Мой телеграм: @eeemmmmlllll